<a href="https://colab.research.google.com/github/JoelFausto/PIBIC_Codes/blob/main/%5BPIBIC%5D_Ataque_de_Infer%C3%AAncia_de_Atributo_Aprendizado_de_M%C3%A1quina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importação das bibliotecas básicas

In [ ]:
!pip install plotly --upgrade

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Ataque de Inferência de Atributo - Abordagens de Aprendizado de Máquina

In [ ]:
# Neste código usaremos técnicas de machine learning para inferir um atributo sensível, como uma doença ou situação socioeconômica, a partir de quase-identificadores.

In [ ]:
pd.set_option('display.max_columns', None)

# Carregar base de dados de microdados anonimizados (exemplo)
microdados = pd.read_parquet('dadosCenso2018_FiltradoPreenchido.parquet')

#### Árvores de Decisão

In [ ]:
# Importando bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

##### Atributo sensível ['IN_FINANCIAMENTO_ESTUDANTIL']

In [ ]:
# Quase-identificadores e o atributo sensível a ser inferido
quase_identificadores_list = [
    microdados[['CO_CURSO']],
    microdados[['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']]
]

atributo_sensivel = microdados[['IN_FINANCIAMENTO_ESTUDANTIL']]

In [ ]:
# Inicializar o DataFrame vazio
resultadosArvores_1 = pd.DataFrame(columns=[
    'Quantidade de quaseidentificadores',
    'Quaseidentificadores',
    'Sucesso Determinístico a Posteriori (%)',
    'Sucesso Probabilístico a Posteriori (%)',
    'Degradação Privacidade Determinística (%)',
    'Degradação Privacidade Probabilística'
])

In [ ]:
# Loop para cada lista completa de quase-identificadores
for quase_identificadores in quase_identificadores_list:
  # Dividir os dados em treino e teste
  X_train, X_test, y_train, y_test = train_test_split(quase_identificadores, atributo_sensivel, test_size=0.3, random_state=42)

  # Treinar o modelo de inferência (Árvore de Decisão)
  modelo_dt = DecisionTreeClassifier()
  modelo_dt.fit(X_train, y_train)

  # Fazer inferência no conjunto de teste
  y_pred = modelo_dt.predict(X_test)
  y_pred_prob = modelo_dt.predict_proba(X_test)[:, 1]  # Probabilidades para AUC-ROC

  # Avaliação do Modelo
  accuracy = accuracy_score(y_test, y_pred) # Cálculo da acurácia (Sucesso a posteriori determinístico)
  roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr') # Cálculo do AUC-ROC (Sucesso a posteriori probabilístico)

  # Calcular Degradação da Privacidade
  baseline_accuracy = max(np.mean(y_test == 1), np.mean(y_test == 0))
  degradacao_privacidade_deterministica = accuracy - baseline_accuracy

  baseline_auc = 0.4728 # Base considerada pelo TED PRICE
  if isinstance(roc_auc, np.ndarray):  # Para o caso multiclasse
      roc_auc_mean = np.mean(roc_auc)  # Calcula a média se for um array
  else:
      roc_auc_mean = roc_auc

  degradacao_privacidade_probabilistica = roc_auc_mean - baseline_auc

  # Adicionar os resultados ao DataFrame
  nova_linha = pd.DataFrame({
      'Quantidade de quaseidentificadores': [len(quase_identificadores.columns)],
      'Quaseidentificadores': [quase_identificadores.columns.tolist()],
      'Sucesso Determinístico a Posteriori (%)': [round(accuracy * 100, 2)],
      'Sucesso Probabilístico a Posteriori (%)': [round(roc_auc * 100, 2)],
      'Degradação Privacidade Determinística (%)': [round(degradacao_privacidade_deterministica * 100, 2)],
      'Degradação Privacidade Probabilística': [degradacao_privacidade_probabilistica]
  })

  resultadosArvores_1 = pd.concat([resultadosArvores_1, nova_linha], ignore_index=True)

In [ ]:
# Aplicar formatação de porcentagem na exibição
resultadosArvores_1 = resultadosArvores_1.style.format({
    'Sucesso Determinístico a Posteriori (%)': '{:.2f}%',
    'Sucesso Probabilístico a Posteriori (%)': '{:.2f}%',
    'Degradação Privacidade Determinística (%)': '{:.2f}%',
})

# Exibir o DataFrame com formatação
resultadosArvores_1

##### Atributo sensível ['IN_DEFICIENCIA']

In [ ]:
# Quase-identificadores e o atributo sensível a ser inferido
quase_identificadores_list = [
    microdados[['CO_CURSO']],
    microdados[['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']]
]

atributo_sensivel = microdados[['IN_DEFICIENCIA']]

In [ ]:
# Inicializar o DataFrame vazio
resultadosArvores_2 = pd.DataFrame(columns=[
    'Quantidade de quaseidentificadores',
    'Quaseidentificadores',
    'Sucesso Determinístico a Posteriori (%)',
    'Sucesso Probabilístico a Posteriori (%)',
    'Degradação Privacidade Determinística (%)',
    'Degradação Privacidade Probabilística'
])

In [ ]:
# Loop para cada lista completa de quase-identificadores
for quase_identificadores in quase_identificadores_list:
  # Dividir os dados em treino e teste
  X_train, X_test, y_train, y_test = train_test_split(quase_identificadores, atributo_sensivel, test_size=0.3, random_state=42)

  # Treinar o modelo de inferência (Árvore de Decisão)
  modelo_dt = DecisionTreeClassifier()
  modelo_dt.fit(X_train, y_train)

  # Fazer inferência no conjunto de teste
  y_pred = modelo_dt.predict(X_test)
  y_pred_prob = modelo_dt.predict_proba(X_test)  # Mantenha todas as colunas

  # Avaliação do Modelo
  accuracy = accuracy_score(y_test, y_pred) # Cálculo da acurácia (Sucesso a posteriori determinístico)

  # Cálculo do AUC-ROC (Sucesso a posteriori probabilístico)
  if len(np.unique(y_test)) == 2:  # Se for um problema binário
      roc_auc = roc_auc_score(y_test, y_pred_prob[:, 1])  # Use apenas a coluna da classe positiva
  else:
      roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')  # Para múltiplas classes

  # Calcular Degradação da Privacidade
  baseline_accuracy = max(np.mean(y_test == 1), np.mean(y_test == 0))  # Maior classe
  degradacao_privacidade_deterministica = accuracy - baseline_accuracy

  baseline_auc = 0.94 # Base considerada pelo TED PRICE
  if isinstance(roc_auc, np.ndarray):  # Para o caso multiclasse
      roc_auc_mean = np.mean(roc_auc)  # Calcula a média se for um array
  else:
      roc_auc_mean = roc_auc

  degradacao_privacidade_probabilistica = roc_auc_mean - baseline_auc

  # Adicionar os resultados ao DataFrame
  nova_linha = pd.DataFrame({
      'Quantidade de quaseidentificadores': [len(quase_identificadores.columns)],
      'Quaseidentificadores': [quase_identificadores.columns.tolist()],
      'Sucesso Determinístico a Posteriori (%)': [round(accuracy * 100, 2)],
      'Sucesso Probabilístico a Posteriori (%)': [round(roc_auc * 100, 2)],
      'Degradação Privacidade Determinística (%)': [round(degradacao_privacidade_deterministica * 100, 2)],
      'Degradação Privacidade Probabilística': [degradacao_privacidade_probabilistica]
  })

  resultadosArvores_2 = pd.concat([resultadosArvores_2, nova_linha], ignore_index=True)

In [ ]:
# Aplicar formatação de porcentagem na exibição
resultadosArvores_2 = resultadosArvores_2.style.format({
    'Sucesso Determinístico a Posteriori (%)': '{:.2f}%',
    'Sucesso Probabilístico a Posteriori (%)': '{:.2f}%',
    'Degradação Privacidade Determinística (%)': '{:.2f}%',
})

# Exibir o DataFrame com formatação
resultadosArvores_2

#### Random Forest

In [ ]:
# Importando bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

##### Atributo sensível ['IN_FINANCIAMENTO_ESTUDANTIL']

In [ ]:
# Quase-identificadores e o atributo sensível a ser inferido
quase_identificadores_list = [
    microdados[['CO_CURSO']],
    microdados[['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']]
]

atributo_sensivel = microdados[['IN_FINANCIAMENTO_ESTUDANTIL']]

In [ ]:
# Inicializar o DataFrame vazio
resultadosRandomForest_1 = pd.DataFrame(columns=[
    'Quantidade de quaseidentificadores',
    'Quaseidentificadores',
    'Sucesso Determinístico a Posteriori (%)',
    'Sucesso Probabilístico a Posteriori (%)',
    'Degradação Privacidade Determinística (%)',
    'Degradação Privacidade Probabilística'
])

In [ ]:
# Loop para cada lista completa de quase-identificadores
for quase_identificadores in quase_identificadores_list:
  # Dividir os dados em treino e teste
  X_train, X_test, y_train, y_test = train_test_split(quase_identificadores, atributo_sensivel, test_size=0.3, random_state=42)

  # Treinar o modelo Random Forest
  modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
  modelo_rf.fit(X_train, y_train)

  # Fazer inferências no conjunto de teste
  y_pred = modelo_rf.predict(X_test)
  y_pred_prob = modelo_rf.predict_proba(X_test)[:, 1] # Probabilidades para AUC-ROC

  # Avaliação do Modelo
  accuracy = accuracy_score(y_test, y_pred) # Cálculo da acurácia (Sucesso a posteriori determinístico)
  roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr') # Cálculo do AUC-ROC (Sucesso a posteriori probabilístico) com suporte a múltiplas classes

  # Calcular Degradação da Privacidade | Acurácia aleatória (modelo de base ingênuo)
  baseline_accuracy = max(np.mean(y_test == 1), np.mean(y_test == 0))  # Maior classe
  degradacao_privacidade_deterministica = accuracy - baseline_accuracy

  baseline_auc = 0.4728 # Base considerada pelo TED PRICE
  if isinstance(roc_auc, np.ndarray):  # Para o caso multiclasse
      roc_auc_mean = np.mean(roc_auc)  # Calcula a média se for um array
  else:
      roc_auc_mean = roc_auc

  degradacao_privacidade_probabilistica = roc_auc_mean - baseline_auc

  # Adicionar os resultados ao DataFrame
  nova_linha = pd.DataFrame({
      'Quantidade de quaseidentificadores': [len(quase_identificadores.columns)],
      'Quaseidentificadores': [quase_identificadores.columns.tolist()],
      'Sucesso Determinístico a Posteriori (%)': [round(accuracy * 100, 2)],
      'Sucesso Probabilístico a Posteriori (%)': [round(roc_auc * 100, 2)],
      'Degradação Privacidade Determinística (%)': [round(degradacao_privacidade_deterministica * 100, 2)],
      'Degradação Privacidade Probabilística': [degradacao_privacidade_probabilistica]
  })

  resultadosRandomForest_1 = pd.concat([resultadosRandomForest_1, nova_linha], ignore_index=True)

In [ ]:
# Aplicar formatação de porcentagem na exibição
resultadosRandomForest_1 = resultadosRandomForest_1.style.format({
    'Sucesso Determinístico a Posteriori (%)': '{:.2f}%',
    'Sucesso Probabilístico a Posteriori (%)': '{:.2f}%',
    'Degradação Privacidade Determinística (%)': '{:.2f}%',
})

# Exibir o DataFrame com formatação
resultadosRandomForest_1

##### Atributo sensível ['IN_DEFICIENCIA']

In [ ]:
# Quase-identificadores e o atributo sensível a ser inferido
quase_identificadores_list = [
    microdados[['CO_CURSO']],
    microdados[['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']]
]

atributo_sensivel = microdados[['IN_DEFICIENCIA']]

In [ ]:
# Inicializar o DataFrame vazio
resultadosRandomForest_2 = pd.DataFrame(columns=[
    'Quantidade de quaseidentificadores',
    'Quaseidentificadores',
    'Sucesso Determinístico a Posteriori (%)',
    'Sucesso Probabilístico a Posteriori (%)',
    'Degradação Privacidade Determinística (%)',
    'Degradação Privacidade Probabilística'
])

In [ ]:
# Loop para cada lista completa de quase-identificadores
for quase_identificadores in quase_identificadores_list:
  # Dividir os dados em treino e teste
  X_train, X_test, y_train, y_test = train_test_split(quase_identificadores, atributo_sensivel, test_size=0.3, random_state=42)

  # Treinar o modelo Random Forest
  modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
  modelo_rf.fit(X_train, y_train)

  # Fazer inferência no conjunto de teste
  y_pred = modelo_rf.predict(X_test)
  y_pred_prob = modelo_rf.predict_proba(X_test)

  # Avaliação do Modelo
  accuracy = accuracy_score(y_test, y_pred) # Cálculo da acurácia (Sucesso a posteriori determinístico)

  # Cálculo do AUC-ROC (Sucesso a posteriori probabilístico)
  if len(np.unique(y_test)) == 2:  # Se for um problema binário
      roc_auc = roc_auc_score(y_test, y_pred_prob[:, 1])  # Use apenas a coluna da classe positiva
  else:
      roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')  # Para múltiplas classes

  # Calcular Degradação da Privacidade
  baseline_accuracy = max(np.mean(y_test == 1), np.mean(y_test == 0))  # Maior classe
  degradacao_privacidade_deterministica = accuracy - baseline_accuracy

  baseline_auc = 0.94 # Base considerada pelo TED PRICE
  if isinstance(roc_auc, np.ndarray):  # Para o caso multiclasse
      roc_auc_mean = np.mean(roc_auc)  # Calcula a média se for um array
  else:
      roc_auc_mean = roc_auc

  degradacao_privacidade_probabilistica = roc_auc_mean - baseline_auc

  # Adicionar os resultados ao DataFrame
  nova_linha = pd.DataFrame({
      'Quantidade de quaseidentificadores': [len(quase_identificadores.columns)],
      'Quaseidentificadores': [quase_identificadores.columns.tolist()],
      'Sucesso Determinístico a Posteriori (%)': [round(accuracy * 100, 2)],
      'Sucesso Probabilístico a Posteriori (%)': [round(roc_auc * 100, 2)],
      'Degradação Privacidade Determinística (%)': [round(degradacao_privacidade_deterministica * 100, 2)],
      'Degradação Privacidade Probabilística': [degradacao_privacidade_probabilistica]
  })

  resultadosRandomForest_2 = pd.concat([resultadosRandomForest_2, nova_linha], ignore_index=True)

In [ ]:
# Aplicar formatação de porcentagem na exibição
resultadosRandomForest_2 = resultadosRandomForest_2.style.format({
    'Sucesso Determinístico a Posteriori (%)': '{:.2f}%',
    'Sucesso Probabilístico a Posteriori (%)': '{:.2f}%',
    'Degradação Privacidade Determinística (%)': '{:.2f}%',
})

# Exibir o DataFrame com formatação
resultadosRandomForest_2

#### Redes Neurais

In [ ]:
# Importando bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from scipy.stats import entropy
from sklearn.preprocessing import label_binarize

##### Atributo sensível ['IN_FINANCIAMENTO_ESTUDANTIL']

In [ ]:
# Quase-identificadores e o atributo sensível a ser inferido
quase_identificadores_list = [
    microdados[['CO_CURSO']],
    microdados[['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']]
]

atributo_sensivel = microdados[['IN_FINANCIAMENTO_ESTUDANTIL']]

In [ ]:
# Inicializar o DataFrame vazio
resultadosRedesNeurais_1 = pd.DataFrame(columns=[
    'Quantidade de quaseidentificadores',
    'Quaseidentificadores',
    'Sucesso Determinístico a Posteriori (%)',
    'Sucesso Probabilístico a Posteriori (%)',
    'Degradação Privacidade Determinística (%)',
    'Degradação Privacidade Probabilística'
])

In [ ]:
# Loop para cada lista completa de quase-identificadores
for quase_identificadores in quase_identificadores_list:
  # Dividir os dados em treino e teste
  X_train, X_test, y_train, y_test = train_test_split(quase_identificadores, atributo_sensivel, test_size=0.3, random_state=42)

  # Normalizar os dados
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Construir a Rede Neural
  modelo = Sequential()
  modelo.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Camada oculta com 64 neurônios
  modelo.add(Dense(32, activation='relu'))  # Outra camada oculta com 32 neurônios
  modelo.add(Dense(1, activation='sigmoid'))  # Saída binária

  # Compilar o modelo
  modelo.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Treinar a rede
  modelo.fit(X_train, y_train, epochs=10, batch_size=10, verbose=1)

  # Fazer inferências no conjunto de teste
  y_pred_prob = modelo.predict(X_test)  # Probabilidades de classe
  y_pred = (y_pred_prob > 0.5).astype("int32")  # Converte probabilidades para classes

  # Avaliação do Modelo
  accuracy = accuracy_score(y_test, y_pred) # Cálculo da acurácia (Sucesso a posteriori determinístico)
  # Cálculo do AUC-ROC (Sucesso a posteriori probabilístico) com suporte a múltiplas classes
  roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

  # Calcular Degradação da Privacidade | Acurácia aleatória (modelo de base ingênuo)
  baseline_accuracy = max(np.mean(y_test == 1), np.mean(y_test == 0))  # Maior classe
  degradacao_privacidade_deterministica = accuracy - baseline_accuracy

  baseline_auc = 0.4728 # Base considerada pelo TED PRICE
  if isinstance(roc_auc, np.ndarray):  # Para o caso multiclasse
      roc_auc_mean = np.mean(roc_auc)  # Calcula a média se for um array
  else:
      roc_auc_mean = roc_auc

  degradacao_privacidade_probabilistica = roc_auc_mean - baseline_auc

  # Adicionar os resultados ao DataFrame
  nova_linha = pd.DataFrame({
      'Quantidade de quaseidentificadores': [len(quase_identificadores.columns)],
      'Quaseidentificadores': [quase_identificadores.columns.tolist()],
      'Sucesso Determinístico a Posteriori (%)': [round(accuracy * 100, 2)],
      'Sucesso Probabilístico a Posteriori (%)': [round(roc_auc_mean * 100, 2)],
      'Degradação Privacidade Determinística (%)': [round(degradacao_privacidade_deterministica * 100, 2)],
      'Degradação Privacidade Probabilística': [degradacao_privacidade_probabilistica]
  })

  resultadosRedesNeurais_1 = pd.concat([resultadosRedesNeurais_1, nova_linha], ignore_index=True)

In [ ]:
# Aplicar formatação de porcentagem na exibição
resultadosRedesNeurais_1 = resultadosRedesNeurais_1.style.format({
    'Sucesso Determinístico a Posteriori (%)': '{:.2f}%',
    'Sucesso Probabilístico a Posteriori (%)': '{:.2f}%',
    'Degradação Privacidade Determinística (%)': '{:.2f}%',
})

# Exibir o DataFrame com formatação
resultadosRedesNeurais_1

##### Atributo sensível ['IN_DEFICIENCIA']

In [ ]:
# Quase-identificadores e o atributo sensível a ser inferido
quase_identificadores_list = [
    microdados[['CO_CURSO']],
    microdados[['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']],
    microdados[['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']]
]

atributo_sensivel = microdados[['IN_DEFICIENCIA']]

In [ ]:
# Inicializar o DataFrame vazio
resultadosRedesNeurais_2 = pd.DataFrame(columns=[
    'Quantidade de quaseidentificadores',
    'Quaseidentificadores',
    'Sucesso Determinístico a Posteriori (%)',
    'Sucesso Probabilístico a Posteriori (%)',
    'Degradação Privacidade Determinística (%)',
    'Degradação Privacidade Probabilística'
])

In [ ]:
# Loop para cada lista completa de quase-identificadores
for quase_identificadores in quase_identificadores_list:
  # Dividir os dados em treino e teste
  X_train, X_test, y_train, y_test = train_test_split(quase_identificadores, atributo_sensivel, test_size=0.3, random_state=42)

  # Normalizar os dados
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Construir a Rede Neural
  modelo = Sequential()
  modelo.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Camada oculta com 64 neurônios
  modelo.add(Dense(32, activation='relu'))  # Outra camada oculta com 32 neurônios
  modelo.add(Dense(1, activation='sigmoid'))  # Saída binária

  # Compilar o modelo
  modelo.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Treinar a rede
  modelo.fit(X_train, y_train, epochs=10, batch_size=10, verbose=1)

  # Fazer inferências no conjunto de teste
  y_pred_prob = modelo.predict(X_test)  # Probabilidades de classe
  y_pred = (y_pred_prob > 0.5).astype("int32")  # Converte probabilidades para classes

  # Avaliação do Modelo
  accuracy = accuracy_score(y_test, y_pred) # Cálculo da acurácia (Sucesso a posteriori determinístico)

  # Cálculo do AUC-ROC (Sucesso a posteriori probabilístico)
  # Verifique se há duas classes em y_test
  n_classes = len(np.unique(y_test))
  if n_classes == 2:
      # AUC-ROC para problema binário
      roc_auc = roc_auc_score(y_test, y_pred_prob)
  else:
      # Binarize y_test para multi-classe e calcular o AUC-ROC
      y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))
      roc_auc = roc_auc_score(y_test_binarized, y_pred_prob, multi_class='ovr')
  # print(f"AUC-ROC (Redes Neurais): {roc_auc:.2f}")

  # Calcular Degradação da Privacidade | Acurácia aleatória (modelo de base ingênuo)
  baseline_accuracy = max(np.mean(y_test == 1), np.mean(y_test == 0))  # Maior classe
  degradacao_privacidade_deterministica = accuracy - baseline_accuracy

  baseline_auc = 0.94 # Base considerada pelo TED PRICE
  if isinstance(roc_auc, np.ndarray):  # Para o caso multiclasse
    roc_auc_mean = np.mean(roc_auc)  # Calcula a média se for um array
  else:
    roc_auc_mean = roc_auc

  degradacao_privacidade_probabilistica = roc_auc - baseline_auc

  # Adicionar os resultados ao DataFrame
  nova_linha = pd.DataFrame({
      'Quantidade de quaseidentificadores': [len(quase_identificadores.columns)],
      'Quaseidentificadores': [quase_identificadores.columns.tolist()],
      'Sucesso Determinístico a Posteriori (%)': [round(accuracy * 100, 2)],
      'Sucesso Probabilístico a Posteriori (%)': [round(roc_auc_mean * 100, 2)],
      'Degradação Privacidade Determinística (%)': [round(degradacao_privacidade_deterministica * 100, 2)],
      'Degradação Privacidade Probabilística': [degradacao_privacidade_probabilistica]
  })

  resultadosRedesNeurais_2 = pd.concat([resultadosRedesNeurais_2, nova_linha], ignore_index=True)

In [ ]:
# Aplicar formatação de porcentagem na exibição
resultadosRedesNeurais_2 = resultadosRedesNeurais_2.style.format({
    'Sucesso Determinístico a Posteriori (%)': '{:.2f}%',
    'Sucesso Probabilístico a Posteriori (%)': '{:.2f}%',
    'Degradação Privacidade Determinística (%)': '{:.2f}%',
})

# Exibir o DataFrame com formatação
resultadosRedesNeurais_2